Currently the script traverses the corpus two times. One time for the generation of the dictionary and then another time to transform the documents in the corpus into a bag of words vector. There is probably a better way to do that.

In [ ]:
import logging
import os.path as path

from treetaggerwrapper import TreeTagger, make_tags
from gensim import corpora, models, similarities

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

corpus_file = path.join('..', 'Crawler', 'Data', 'Corpus.txt')
stopwords_file = path.join('..', 'Crawler', 'Data', 'Stopwords.txt')
wanted_tags = set(['NN', 'VVPP', 'ADJD'])

class StreamCorpus(object):
    def __init__(self, filename):
        self.filename = filename
    def __iter__(self):
        for document in open(self.filename, encoding='UTF-8'):
            keywords = extract_keywords(document)
            yield dictionary.doc2bow(keywords)

def extract_keywords(document):
    tagged = tagger.tag_text(document)
    tags = make_tags(tagged, exclude_nottags=True)
    keywords = [lemma.lower() for word, tag, lemma in tags if tag in wanted_tags]
    return keywords

def load_stopwords(filename):
    with open(filename, 'r', encoding='UTF-8') as f:
        data = f.read()
        return set(data.split('\n'))

corpus = StreamCorpus(corpus_file)
stoplist = load_stopwords(stopwords_file)

tagger = TreeTagger(TAGLANG='de')

dictionary = corpora.Dictionary(extract_keywords(document) for document in open(corpus_file, encoding='UTF-8'))
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist
                                          if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid, docfreq in dictionary.dfs.items() if docfreq == 1]
dictionary.filter_tokens(stop_ids + once_ids)
dictionary.filter_extremes(no_below=10)
dictionary.compactify()

dictionary.save('corpus.dict')
corpora.MmCorpus.serialize('corpus.mm', corpus)